In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.regularizers import l2
import pandas as pd
import numpy as np
import ast
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras.initializers import RandomNormal
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import MultiHeadAttention, GlobalAveragePooling1D

In [6]:
def build_fall_detection_model_multiattn(timesteps, features, lstm_units=64, dropout_rate=0.2,
                                         dense_units=34, l2_reg=0.01, num_classes=2,
                                         num_heads=4, key_dim=16):
    """
    Construye un modelo RNN con MultiHeadAttention y GlobalAveragePooling1D.

    Args:
        timesteps (int): Número de pasos de tiempo en cada secuencia.
        features (int): Número de características por paso de tiempo.
        lstm_units (int): Unidades de la LSTM.
        dropout_rate (float): Tasa de dropout.
        dense_units (int): Unidades en la capa densa intermedia.
        l2_reg (float): Regularización L2.
        num_classes (int): Número de clases de salida.
        num_heads (int): Número de "cabezas" de atención.
        key_dim (int): Dimensión de las claves por cabeza.

    Returns:
        keras.Model: Modelo compilado.
    """
    input_shape = (timesteps, features)
    inputs = layers.Input(shape=input_shape)

    # 1. Masking
    masked_inputs = layers.Masking(mask_value=0.0)(inputs)

    # 2. LSTM
    lstm_out = layers.LSTM(lstm_units, return_sequences=True)(masked_inputs)

    # 3. Dropout
    dropout_out = layers.Dropout(dropout_rate)(lstm_out)

    # 4. MultiHead Attention
    attn_out = MultiHeadAttention(num_heads=num_heads, key_dim=key_dim)(dropout_out, dropout_out)

    # 5. GlobalAveragePooling
    pooled_out = GlobalAveragePooling1D()(attn_out)

    # 6. Dense
    dense_out = layers.Dense(dense_units, activation='relu',
                             kernel_initializer=RandomNormal(),
                             kernel_regularizer=l2(l2_reg))(pooled_out)

    # 7. Output
    outputs = layers.Dense(num_classes, activation='softmax')(dense_out)

    model = Model(inputs=inputs, outputs=outputs)
    return model

def train_model(model, X_train, y_train, X_val=None, y_val=None,
                epochs=50, batch_size=32, validation_split=0.2,
                patience=10):
    """
    Compila y entrena el modelo Keras.
    (Misma función que antes)
    """
    # Compilar el modelo
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy', # Confirmado que 'y' es one-hot (2 cols)
                  metrics=['accuracy'])

    # Callback para detener el entrenamiento si no mejora
    early_stopping = EarlyStopping(monitor='val_loss',
                                   patience=patience,
                                   restore_best_weights=True)

    # Determinar datos de validación
    if X_val is not None and y_val is not None:
        validation_data = (X_val, y_val)
        val_split = 0.0
    else:
        validation_data = None
        val_split = validation_split

    print(f"\nIniciando entrenamiento por {epochs} épocas...")

    # Entrenar el modelo
    history = model.fit(X_train, y_train,
                        epochs=epochs,
                        batch_size=batch_size,
                        validation_split=val_split,
                        validation_data=validation_data,
                        callbacks=[early_stopping],
                        verbose=1)

    print("Entrenamiento completado.")
    return history

In [8]:
df = pd.read_csv('combined_padded.csv')

# Asegúrate de que los campos 'features' y 'poses_sec' son listas
def safe_literal_eval(val):
    # Maneja valores nulos/faltantes devolviendo lista vacía o None según prefieras
    if pd.isna(val):
        return [] # O manejar de otra forma si es necesario
    if isinstance(val, str):
        try:
            return ast.literal_eval(val)
        except (ValueError, SyntaxError):
            print(f"Advertencia: No se pudo evaluar: {val}")
            return [] # O manejar de otra forma
    return val # Si ya es una lista/otro tipo

df['features'] = df['features'].apply(safe_literal_eval)
df['poses_sec'] = df['poses_sec'].apply(safe_literal_eval)

# -------------------------
# 2. Combinar features + landmarks por frame
# -------------------------
sequences = []
max_len_actual = 0 # Para verificar la longitud si es necesario
for i, (feats, lms) in enumerate(zip(df['features'], df['poses_sec'])):
    # Añadir una verificación por si una de las listas está vacía
    if not feats or not lms:
        print(f"Advertencia: Lista vacía encontrada en índice {i}. Saltando muestra.")
        continue # O manejar añadiendo ceros si es apropiado

    # Verifica si las longitudes coinciden antes de hacer zip
    if len(feats) != len(lms):
        print(f"Advertencia: Longitudes no coinciden en índice {i}. feats: {len(feats)}, lms: {len(lms)}. Usando la longitud menor.")
        min_len = min(len(feats), len(lms))
        feats = feats[:min_len]
        lms = lms[:min_len]

    combined_seq = [f + l for f, l in zip(feats, lms)]
    #print(combined_seq)
    if not combined_seq: # Si después del zip sigue vacío
        print(f"Advertencia: Secuencia combinada vacía en índice {i}. Saltando.")
        continue
    sequences.append(combined_seq)
    max_len_actual = max(max_len_actual, len(combined_seq))

# Asegúrate de que todas las secuencias tengan la misma longitud (Padding ya debería estar hecho)
# Si el padding no se hizo antes o fue inconsistente, aquí es donde fallaría np.array()
print(f"Longitud máxima encontrada en secuencias combinadas: {max_len_actual}") 
# Debería ser 129 según tus datos originales.

# Convertir a array NumPy. Si falla aquí, el padding previo tuvo problemas.
try:
    X = np.array(sequences, dtype=np.float32) # Especificar dtype puede ayudar
except ValueError as e:
    print(f"Error al convertir a NumPy array: {e}")
    print("Esto usualmente indica que las secuencias internas tienen longitudes diferentes.")
    # Aquí podrías añadir código para forzar el padding si es necesario,
    # pero basado en tu archivo 'combined_padded.csv', no debería ser necesario.
    # Ejemplo de padding si fuera necesario:
    # from tensorflow.keras.preprocessing.sequence import pad_sequences
    # X = pad_sequences(sequences, maxlen=129, padding='post', truncating='post', dtype='float32')


# -------------------------
# 3. Preparar las etiquetas
# -------------------------
y_labels = df['target'].values # Asegúrate que la columna se llama 'target'
encoder = LabelBinarizer()
y = encoder.fit_transform(y_labels)

if y.shape[1] == 1:
    y = np.hstack((1 - y, y))

# Eliminar filas de 'y' donde se saltaron secuencias en 'X' (si se hizo 'continue')
# Esto requiere rastrear los índices saltados o re-filtrar df antes de obtener y_labels
# Por simplicidad, asumimos que no se saltaron filas por ahora.

# -------------------------
# 4. Separar en train/test
# -------------------------
# Asegúrate que X y y tengan el mismo número de muestras antes de dividir
if X.shape[0] != y.shape[0]:
    raise ValueError(f"Discrepancia en número de muestras: X tiene {X.shape[0]}, y tiene {y.shape[0]}")

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y) # Añadir stratify es bueno para clasificación

print("Formas finales de los datos:")
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

Longitud máxima encontrada en secuencias combinadas: 130
Formas finales de los datos:
(118, 130, 109) (30, 130, 109) (118, 2) (30, 2)


In [10]:
if X_train.size == 0:
    print("Error: X_train está vacío después del preprocesamiento.")
else:
    num_samples, timesteps, features = X_train.shape
    num_classes = y_train.shape[1]

    # Construir el modelo con Attention
    fall_model_attn = build_fall_detection_model_multiattn(timesteps=timesteps,
                                                                features=features,
                                                                num_classes=num_classes)

    # Entrenar el modelo
    history = train_model(fall_model_attn, X_train, y_train, epochs=100, batch_size=16, validation_split=0.1)

    # Evaluar después
    print("\nEvaluando el modelo (con Attention) en el conjunto de prueba:")
    loss, accuracy = fall_model_attn.evaluate(X_test, y_test, verbose=0)
    print(f"Pérdida en Prueba: {loss:.4f}")
    print(f"Precisión en Prueba: {accuracy:.4f}")


Iniciando entrenamiento por 100 épocas...
Epoch 1/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 4s 97ms/step - accuracy: 0.3894 - loss: 0.7488 - val_accuracy: 0.7333 - val_loss: 0.7287
Epoch 2/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - accuracy: 0.5044 - loss: 0.7308 - val_accuracy: 0.5333 - val_loss: 0.7125
Epoch 3/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - accuracy: 0.5002 - loss: 0.7240 - val_accuracy: 0.5333 - val_loss: 0.6901
Epoch 4/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - accuracy: 0.5255 - loss: 0.7059 - val_accuracy: 0.6333 - val_loss: 0.6490
Epoch 5/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - accuracy: 0.6164 - loss: 0.6704 - val_accuracy: 0.8000 - val_loss: 0.5868
Epoch 6/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - accuracy: 0.7209 - loss: 0.6150 - val_accuracy: 0.7667 - val_loss: 0.5199
Epoch 7/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - accuracy: 0.6816 - loss: 0.5969 - val_accuracy: 0.8000 - val_loss: 0.4662
Epoch 8/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - accuracy: 0.7856 - loss

In [11]:
fall_model_attn.save('fall_detection_model.keras')